In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:

from pathlib import Path

import flwr
import torch
from flwr.simulation import run_simulation
from torch.optim.lr_scheduler import CosineAnnealingLR

from fl_g13.architectures import BaseDino
from fl_g13.editing import SparseSGDM
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.server_app import get_server_app
from fl_g13.fl_pytorch.editing.centralized_mask import save_mask
from fl_g13.fl_pytorch.editing.centralized_mask import load_mask

2025-06-03 20:15:06.146 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


In [4]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")
# disable_progress_bar()

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


# Login wandb

In [5]:
!pip install wandb


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
## read .env file
import dotenv

dotenv.load_dotenv()


True

In [7]:
import wandb

# login by key in .env file
WANDB_API_KEY = dotenv.dotenv_values()["WANDB_API_KEY"]
wandb.login(key=WANDB_API_KEY)

C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\.venv\lib\site-packages\notebook\utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\ADMIN\_netrc
wandb: Currently logged in as: thanhnv-it23 (stefano-gamba-social-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## Build module local

Build module local such that ClientApp can use it

In [8]:
!pip install -e ..

Obtaining file:///C:/Users/ADMIN/Desktop/BACKUP/study/Italy/polito/classes/20242/deep%20learning/project/source_code/fl-g13


  Installing build dependencies: started

  Installing build dependencies: finished with status 'done'


  Checking if build backend supports build_editable: started


[notice] A new release of pip is available: 25.0.1 -> 25.1.1


[notice] To update, run: python.exe -m pip install --upgrade pip

  Checking if build backend supports build_editable: finished with status 'done'


  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for fl_g13 (pyproject.toml): started
  Building editable for fl_g13 (pyproject.toml): finished with status 'done'
  Created wheel for fl_g13: filename=fl_g13-0.0.1-py3-none-any.whl size=4847 sha256=3d4a704acfba3084c75fba927a146db01eabf4c77ae16fac05a23835dca65b2b


### Download missing module for clients

Dino model,that is serialized and sent to client by server, require some modules that have to download from source code of dino model


In [9]:
import os
import urllib.request


def download_if_not_exists(file_path: str, file_url: str):
    """
    Checks if a file exists at the given path. If it does not, downloads it from the specified URL.

    Parameters:
    - file_path (str): The local path to check and save the file.
    - file_url (str): The URL from which to download the file.
    """
    if not os.path.exists(file_path):
        print(f"'{file_path}' not found. Downloading from {file_url}...")
        try:
            urllib.request.urlretrieve(file_url, file_path)
            print("Download complete.")
        except Exception as e:
            print(f"Failed to download file: {e}")
    else:
        print(f"'{file_path}' already exists.")

In [10]:
download_if_not_exists("vision_transformer.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/vision_transformer.py")
download_if_not_exists("utils.py",
                       "https://raw.githubusercontent.com/facebookresearch/dino/refs/heads/main/utils.py")


'vision_transformer.py' already exists.
'utils.py' already exists.


# FL

## Configs

In [11]:

from pathlib import Path

import flwr
import torch
from flwr.simulation import run_simulation
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import DataLoader
from torchvision import datasets

from fl_g13.architectures import BaseDino
from fl_g13.config import RAW_DATA_DIR
from fl_g13.editing import SparseSGDM
from fl_g13.fl_pytorch.client_app import get_client_app
from fl_g13.fl_pytorch.datasets import get_eval_transforms
from fl_g13.fl_pytorch.server_app import get_server_app
from fl_g13.modeling.eval import eval

In [12]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = "cpu"
print(f"Training on {DEVICE}")
print(f"Flower {flwr.__version__} / PyTorch {torch.__version__}")

Training on cuda
Flower 1.17.0 / PyTorch 2.6.0+cu118


In [13]:
DEBUG = False

In [14]:
# Model config

## Model Hyper-parameters
head_layers = 3
head_hidden_size = 512
dropout_rate = 0.0
unfreeze_blocks = 1

## Training Hyper-parameters
batch_size = 128
lr = 1e-3
momentum = 0.9
weight_decay = 1e-5
T_max = 8
eta_min = 1e-5

# FL config
K = 100
C = 0.1
J = 4
num_rounds = 30
partition_type = 'iid'

## only for partition_type = 'shard'
num_shards_per_partition = 10

## Server App config
save_every = 1
fraction_fit = C  # Sample of available clients for training
fraction_evaluate = 0.1  # Sample 50% of available clients for evaluation
min_fit_clients = 10  # Never sample less than 10 clients for training
min_evaluate_clients = 5  # Never sample less than 5 clients for evaluation
min_available_clients = 10  # Wait until all 10 clients are available
device = 'cuda'
## checkpoints directory
current_path = Path.cwd()
model_save_path = current_path / f"../models/fl_dino_baseline/noniid"
checkpoint_dir = model_save_path.resolve()
os.makedirs(checkpoint_dir, exist_ok=True)

## Wandb config
use_wandb = True
wandb_config = {
    # wandb param
    'name': 'FL_Dino_Baseline_iid',
    'project_name': "FL_test_chart",
    # model config param
    "fraction_fit": fraction_fit,
    "lr": lr,
    "momentum": momentum,
    'partition_type': partition_type,
    'K': K,
    'C': C,
    'J': J,
}

# model editing config
model_editing = True
mask_type = 'global'
sparsity = 0.2
mask = None

## simulation run config
NUM_CLIENTS = 100
MAX_PARALLEL_CLIENTS = 10

if DEBUG:
    use_wandb = False
    num_rounds = 2
    J = 4


## Define model , optimizer and loss function

In [25]:
from fl_g13.modeling import load_or_create

# Model
model, start_epoch = load_or_create(
    path=checkpoint_dir,
    model_class=BaseDino,
    model_config=None,
    optimizer=None,
    scheduler=None,
    device=device,
    verbose=True,
)

model.to(DEVICE)

unfreeze_blocks=12
model.unfreeze_blocks(unfreeze_blocks)
# optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)

# Create a dummy mask for SparseSGDM
init_mask = [torch.ones_like(p, device=p.device) for p in
             model.parameters()]  # Must be done AFTER the model is moved to the device
# Optimizer, scheduler, and loss function
optimizer = SparseSGDM(
    model.parameters(),
    mask=init_mask,
    lr=lr,
    momentum=0.9,
    weight_decay=1e-5
)
criterion = torch.nn.CrossEntropyLoss()
scheduler = CosineAnnealingLR(
    optimizer=optimizer,
    T_max=T_max,
    eta_min=eta_min
)

🔍 Loading checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_dino_baseline\noniid\fl_fl_baseline_BaseDino_epoch_200_noniid_1_4.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 0, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main
Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_dino_baseline\noniid\fl_fl_baseline_BaseDino_epoch_200_noniid_1_4.pth, resuming at epoch 201


## Calculate the centralized mask

In [26]:
from fl_g13.fl_pytorch.editing.centralized_mask import get_centralized_mask

## config client data set params
client_partition_type = 'shard'  # 'iid' or 'shard' for non-iid dataset
client_num_partitions = 100  # equal to number of client
client_num_shards_per_partition = 10
client_batch_size = 16
client_train_test_split_ratio = 0.2
client_dataset = "cifar100"
client_seed = 42,
client_return_dataset = False,

## config get mask params
mask_model = model
mask_sparsity = 0.8
mask_type = 'global'
mask_rounds = 1
mask_func = None

## aggregate
agg_strategy = 'union'
agg_func = None

if DEBUG:
    client_num_partitions = 10
    client_batch_size = 128
    client_train_test_split_ratio = 0.9


In [21]:
from typing import Dict, Any

def compute_mask_stats(mask_dict: Dict[str, torch.Tensor]) -> Dict[str, Any]:
    """
    Computes various statistics for a mask represented as a dictionary
    mapping parameter names to mask tensors.

    Args:
        mask_dict: A dictionary where keys are parameter names (str)
                   and values are mask tensors (torch.Tensor) containing 0s and 1s.

    Returns:
        A dictionary containing overall and layer-wise mask statistics.
    """
    stats = {}

    # --- Overall Statistics ---
    total_elements = 0
    kept_elements_overall = 0 # Elements with value 1
    masked_elements_overall = 0 # Elements with value 0

    for name, mask_tensor in mask_dict.items():
        num_elements = mask_tensor.numel()
        kept_in_layer = torch.sum(mask_tensor == 1).item()
        masked_in_layer = num_elements - kept_in_layer

        total_elements += num_elements
        kept_elements_overall += kept_in_layer
        masked_elements_overall += masked_in_layer

        # --- Layer-wise Statistics ---
        layer_stats = {
            'num_elements': num_elements,
            'kept_elements': kept_in_layer,
            'masked_elements': masked_in_layer,
            'density': kept_in_layer / num_elements if num_elements > 0 else 0.0,
            'sparsity': masked_in_layer / num_elements if num_elements > 0 else 0.0
        }
        stats[name] = layer_stats

    # --- Add Overall Statistics to the result dictionary ---
    stats['overall'] = {
        'total_elements': total_elements,
        'kept_elements': kept_elements_overall,
        'masked_elements': masked_elements_overall,
        'density': kept_elements_overall / total_elements if total_elements > 0 else 0.0,
        'sparsity': masked_elements_overall / total_elements if total_elements > 0 else 0.0
    }

    return stats


def print_mask_stats(stats: Dict[str, Any], layer = False):
    """
    Prints the mask statistics in a readable format.

    Args:
        stats: The dictionary returned by compute_mask_stats.
    """
    if 'overall' not in stats:
        print("Invalid stats dictionary format.")
        return

    overall_stats = stats['overall']
    print("--- Overall Mask Statistics ---")
    print(f"Total Elements: {overall_stats['total_elements']}")
    print(f"Kept Elements (1s): {overall_stats['kept_elements']}")
    print(f"Masked Elements (0s): {overall_stats['masked_elements']}")
    print(f"Overall Density: {overall_stats['density']:.4f}")
    print(f"Overall Sparsity: {overall_stats['sparsity']:.4f}")
    print("-" * 30)

    if not layer:
      return

    print("--- Layer-wise Mask Statistics ---")
    # Sort layer names for consistent output
    layer_names = sorted([name for name in stats if name != 'overall'])
    for name in layer_names:
        layer_stats = stats[name]
        print(f"Layer: {name}")
        print(f"  Num Elements: {layer_stats['num_elements']}")
        print(f"  Kept Elements: {layer_stats['kept_elements']}")
        print(f"  Masked Elements: {layer_stats['masked_elements']}")
        print(f"  Density: {layer_stats['density']:.4f}")
        print(f"  Sparsity: {layer_stats['sparsity']:.4f}")
        print("-" * 20)

In [27]:


centralized_mask = get_centralized_mask(
    client_partition_type=client_partition_type,
    client_num_partitions=client_num_partitions,
    client_num_shards_per_partition=client_num_shards_per_partition,
    client_batch_size=client_batch_size,
    client_dataset=client_dataset,
    client_seed=client_seed,
    client_return_dataset=client_return_dataset,
    mask_model=mask_model,
    mask_sparsity=mask_sparsity,
    mask_type=mask_type,
    mask_rounds=mask_rounds,
    mask_func=mask_func,
    agg_strategy=agg_strategy,
    agg_func=agg_func
)


Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


Fisher Score: 100%|██████████| 25/25 [00:02<00:00,  8.55batch/s]


	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


Fisher Score: 100%|██████████| 25/25 [00:02<00:00,  9.73batch/s]


	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


...

	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


Fisher Score: 100%|██████████| 25/25 [00:04<00:00,  5.57batch/s]


	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


Fisher Score: 100%|██████████| 25/25 [00:04<00:00,  5.75batch/s]


	Updating the mask


In [33]:
# ## save and load mask to/from pth file
# save_mask(centralized_mask, filepath=f"centralized_mask.pth")
# mask = load_mask(filepath=f"centralized_mask.pth")
# mask

[tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [ ]:

agg_strategy = 'intersection'
centralized_mask_intersection = get_centralized_mask(
    client_partition_type=client_partition_type,
    client_num_partitions=client_num_partitions,
    client_num_shards_per_partition=client_num_shards_per_partition,
    client_batch_size=client_batch_size,
    client_dataset=client_dataset,
    client_seed=client_seed,
    client_return_dataset=client_return_dataset,
    mask_model=mask_model,
    mask_sparsity=mask_sparsity,
    mask_type=mask_type,
    mask_rounds=mask_rounds,
    mask_func=mask_func,
    agg_strategy=agg_strategy,
    agg_func=agg_func
)

Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


Fisher Score: 100%|██████████| 25/25 [00:02<00:00,  8.67batch/s]


	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


Fisher Score: 100%|██████████| 25/25 [00:02<00:00,  9.47batch/s]


	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


...

	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


Fisher Score: 100%|██████████| 25/25 [00:02<00:00,  9.22batch/s]


	Updating the mask
Computing simple global mask with target sparsity (0.80).
Round 1/1.
	Current round density 0.20%
	Computing the masked fisher score


Fisher Score: 100%|██████████| 25/25 [00:02<00:00,  9.58batch/s]


	Updating the mask


In [28]:
compute_mask_stats(centralized_mask[1])

{'backbone.blocks.0.norm1.weight': {'num_elements': 384,
  'kept_elements': 300,
  'masked_elements': 84,
  'density': 0.78125,
  'sparsity': 0.21875},
 'backbone.blocks.0.norm1.bias': {'num_elements': 384,
  'kept_elements': 48,
  'masked_elements': 336,
  'density': 0.125,
  'sparsity': 0.875},
 'backbone.blocks.0.attn.qkv.weight': {'num_elements': 442368,
  'kept_elements': 414388,
  'masked_elements': 27980,
  'density': 0.9367494936342593,
  'sparsity': 0.06325050636574074},
 'backbone.blocks.0.attn.qkv.bias': {'num_elements': 1152,
  'kept_elements': 743,
  'masked_elements': 409,
  'density': 0.6449652777777778,
  'sparsity': 0.3550347222222222},
 'backbone.blocks.0.attn.proj.weight': {'num_elements': 147456,
  'kept_elements': 93482,
  'masked_elements': 53974,
  'density': 0.6339653862847222,
  'sparsity': 0.3660346137152778},
 'backbone.blocks.0.attn.proj.bias': {'num_elements': 384,
  'kept_elements': 0,
  'masked_elements': 384,
  'density': 0.0,
  'sparsity': 1.0},
 'backb

In [24]:
compute_mask_stats(centralized_mask_intersection[1])

{'head.0.weight': {'num_elements': 196608,
  'kept_elements': 159,
  'masked_elements': 196449,
  'density': 0.0008087158203125,
  'sparsity': 0.9991912841796875},
 'head.0.bias': {'num_elements': 512,
  'kept_elements': 30,
  'masked_elements': 482,
  'density': 0.05859375,
  'sparsity': 0.94140625},
 'head.3.weight': {'num_elements': 262144,
  'kept_elements': 5025,
  'masked_elements': 257119,
  'density': 0.019168853759765625,
  'sparsity': 0.9808311462402344},
 'head.3.bias': {'num_elements': 512,
  'kept_elements': 22,
  'masked_elements': 490,
  'density': 0.04296875,
  'sparsity': 0.95703125},
 'head.6.weight': {'num_elements': 262144,
  'kept_elements': 16120,
  'masked_elements': 246024,
  'density': 0.061492919921875,
  'sparsity': 0.938507080078125},
 'head.6.bias': {'num_elements': 512,
  'kept_elements': 25,
  'masked_elements': 487,
  'density': 0.048828125,
  'sparsity': 0.951171875},
 'head.9.weight': {'num_elements': 51200,
  'kept_elements': 1629,
  'masked_elements'

## Define the Client, Server Apps

In [34]:
client = get_client_app(
    model=model,
    optimizer=optimizer,
    criterion=criterion,
    device=DEVICE,
    partition_type=partition_type,
    local_epochs=J,
    batch_size=batch_size,
    num_shards_per_partition=num_shards_per_partition,
    scheduler=scheduler,
    verbose=0,
    model_editing=model_editing,
    mask_type=mask_type,
    sparsity=sparsity,
    mask=mask
)

In [35]:
server = get_server_app(checkpoint_dir=checkpoint_dir,
                        model_class=model,
                        optimizer=optimizer,
                        criterion=criterion,
                        scheduler=scheduler,
                        num_rounds=num_rounds,
                        fraction_fit=fraction_fit,
                        fraction_evaluate=fraction_evaluate,
                        min_fit_clients=min_fit_clients,
                        min_evaluate_clients=min_evaluate_clients,
                        min_available_clients=min_available_clients,
                        device=device,
                        use_wandb=use_wandb,
                        wandb_config=wandb_config,
                        save_every=save_every,
                        prefix='fl_baseline'
                        )

🔍 Loading checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_dino_baseline\iid\fl_fl_baseline_BaseDino_epoch_83.pth
📦 Model class in checkpoint: BaseDino
🔧 Model configuration: {'variant': 'dino_vits16', 'dropout_rate': 0.0, 'head_hidden_size': 512, 'head_layers': 3, 'num_classes': 100, 'unfreeze_blocks': 1, 'activation_fn': 'GELU', 'pretrained': True}


Using cache found in C:\Users\ADMIN/.cache\torch\hub\facebookresearch_dino_main


➡️ Moved model to device: cuda
✅ Loaded checkpoint from C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13\models\fl_dino_baseline\iid\fl_fl_baseline_BaseDino_epoch_83.pth, resuming at epoch 84


## Before training

Test model performance before fine-turning

In [36]:
# testset = datasets.CIFAR100(RAW_DATA_DIR, train=False, download=True, transform=get_eval_transforms())
# testloader = DataLoader(testset, batch_size=32)

In [37]:
# test_loss, test_accuracy, _ = eval(testloader, model, criterion)
# test_loss, test_accuracy

## Run the training


In [38]:
# Specify the resources each of your clients need
# By default, each client will be allocated 1x CPU and 0x GPUs
backend_config = {"client_resources": {"num_cpus": 1, "num_gpus": 0.0}}

# When running on GPU, assign an entire GPU for each client
if DEVICE == "cuda":
    backend_config["client_resources"] = {"num_cpus": 1, "num_gpus": 1}
    # Refer to our Flower framework documentation for more details about Flower simulations
    # and how to set up the `backend_config`

In [39]:
# Run simulation
run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=NUM_CLIENTS,
    backend_config=backend_config
)

[Server] Server on device: cuda:0
[Server] CUDA available in client: True
Using strategy 'CustomFedAvg' (default option)


INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Starting evaluation of initial global parameters


[Server Eval Round 0] Model device: cuda:0
[Server Eval Round 0] CUDA available in server eval: True


Eval progress: 100%|██████████| 313/313 [00:35<00:00,  8.73batch/s]
INFO :      [Round 0] Centralized Evaluation - Loss: 0.8345, Metrics: {'centralized_accuracy': 0.7662}
INFO :      initial parameters (loss, other metrics): 0.8344755021337503, {'centralized_accuracy': 0.7662}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 10 clients (out of 100)
(ClientAppActor pid=25544) 2025-05-28 17:39:20.940 | INFO     | fl_g13.config:<module>:11 - PROJ_ROOT path is: C:\Users\ADMIN\Desktop\BACKUP\study\Italy\polito\classes\20242\deep learning\project\source_code\fl-g13


(ClientAppActor pid=25544) [Client] Client on device: cuda:0
(ClientAppActor pid=25544) [Client] CUDA available in client: True
(ClientAppActor pid=25544) No prefix/name for the model was provided, choosen prefix/name: wacky_charizard_38
(ClientAppActor pid=25544) 


...

Eval progress: 100%|██████████| 1/1 [00:00<00:00,  2.81batch/s]


(ClientAppActor pid=25544) [Client] Client on device: cuda:0
(ClientAppActor pid=25544) [Client] CUDA available in client: True


Eval progress: 100%|██████████| 1/1 [00:00<00:00,  3.18batch/s]


(ClientAppActor pid=25544) [Client] Client on device: cuda:0
(ClientAppActor pid=25544) [Client] CUDA available in client: True


Eval progress:   0%|          | 0/1 [00:00<?, ?batch/s]
INFO :      aggregate_evaluate: received 10 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 2 round(s) in 267.51s
INFO :      	History (loss, distributed):
INFO :      		round 1: 0.9110377073287964
INFO :      		round 2: 0.8412152230739594
INFO :      	History (loss, centralized):
INFO :      		round 0: 0.8344755021337503
INFO :      		round 1: 0.8339574392230366
(ClientAppActor pid=25544)INFO :      		round 2: 0.8335269415340485
Eval progress: 100%|██████████| 1/1 [00:00<00:00,  3.25batch/s]
INFO :      	History (metrics, distributed, fit):
INFO :      	{'avg_drift': [(1, np.float32(0.00013432844)), (2, np.float32(0.00012472943))],
INFO :      	 'avg_train_loss': [(1, 0.5474612358957529), (2, 0.5060249358415604)],
INFO :      	 'relative_drift': [(1, np.float32(2.5880007e-07)),
INFO :      	                    (2, np.float32(2.403064e-07))]}
INFO :      	History (metrics, distributed, evaluate):